# Lesson 3: Updating Records by ID

# Updating Records by ID

Welcome to the section on updating records by ID using Django ORM. In our previous lessons, we learned how to retrieve records from a database. Now, we'll advance our skills and focus on updating specific records. This is crucial for any application that requires data modification, such as editing user profiles, updating products, or managing tasks.

## What You'll Learn
In this lesson, you'll discover how to update a specific record in the database using its ID. This allows you to change the details of an item, which is a common requirement in many applications.

We will continue using the **Todo** model from our earlier lessons:

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

To update a `Todo` item by its ID, we'll add a new view function in `views.py`. This function will handle **PUT** requests to update the `task` field of the `Todo` object. Here’s the code snippet:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.views.decorators.http import require_http_methods
from django.shortcuts import get_object_or_404
from .models import Todo
import json

@csrf_exempt
@require_http_methods(['PUT'])
def update_todo(request, id):
    try:
        todo = get_object_or_404(Todo, id=id)
        data = json.loads(request.body)
        todo.task = data['task']
        todo.save()
        return JsonResponse({'message': 'Todo updated'})
    except Exception as e:
        return JsonResponse({'error': str(e)}, status=400)
```

This function:
1. Ensures that only **PUT** requests are accepted.
2. Retrieves the `Todo` object by its ID using `get_object_or_404`.
3. Updates its `task` field with new data from the request body.
4. Saves the changes to the database.
5. Returns a JSON response with a success message if the update is successful.
6. If any error occurs, it returns an error message with a `400` status code.

## Why It Matters
Knowing how to update records is a fundamental skill in web development. Here are some scenarios where this knowledge is indispensable:

- **Editing User Profiles**: Websites often allow users to update their profile information such as name, email, or address.
- **Managing Products**: In e-commerce platforms, administrators need to update product details like price, description, and availability.
- **Task Management**: To-do list applications often require functionality to update the details of tasks.

By mastering record updates, you will make your applications more interactive and user-friendly. Users expect to be able to modify data effortlessly, and your skills will ensure they can do so seamlessly.

## Ready to Practice?
Excited to dive in? Let's proceed to the practice section and start applying what we've learned. This hands-on experience will cement your understanding and boost your confidence in updating records by ID.

Happy coding! 😊

## Update Records with Django ORM

To execute the code successfully, here’s an overview of each component and how it fits into the process:

### 1. **Model Definition**

The `Todo` model defines the structure for storing each task. It includes:
- `task`: A description of the task.
- `completed`: A Boolean indicating whether the task is completed.

```python
from django.db import models

class Todo(models.Model):
    task = models.CharField(max_length=200)
    completed = models.BooleanField(default=False)

    def __str__(self):
        return self.task
```

### 2. **View Functions**

- **`preload`**: This view preloads some data into the `Todo` model, creating tasks if they don't already exist.
- **`update_todo`**: This view updates a specific task by its ID, modifying the `task` field based on the data received via the PUT request.
- **`get_todos`**: This view retrieves all the tasks from the database and returns them as a JSON response.

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.views.decorators.http import require_http_methods
from django.shortcuts import get_object_or_404
from .models import Todo
import json

@csrf_exempt
def preload(request):
    todos = [('Buy milk', False), ('Buy eggs', False), ('Buy bread', True)]
    for todo in todos:
        if not Todo.objects.filter(task=todo[0]).exists():
            Todo.objects.create(task=todo[0], completed=todo[1])
    return JsonResponse({'message': 'Preloaded todos'})

@csrf_exempt
@require_http_methods(['PUT'])
def update_todo(request, id):
    try:
        todo = get_object_or_404(Todo, pk=id)
        data = json.loads(request.body)
        todo.task = data['task']
        todo.save()
        return JsonResponse({'message': 'Todo updated'})
    except Exception as e:
        return JsonResponse({'error': str(e)}, status=400)

@csrf_exempt
def get_todos(request):
    todos = Todo.objects.all()
    return JsonResponse({'todos': [{'id': todo.id, 'task': todo.task, 'completed': todo.completed} for todo in todos]})
```

### 3. **URL Configuration**

Define the URL patterns that map to each view. These allow you to access the views through specific endpoints:
- `/preload/`: Preload data.
- `/todos/<int:id>/`: Update a task by ID using a PUT request.
- `/todos/`: Retrieve all tasks.

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('preload/', views.preload, name='preload'),
    path('todos/<int:id>/', views.update_todo, name='update_todo'),
    path('todos/', views.get_todos, name='get_todos'),
]
```

### 4. **Sending Requests**

Use Python's `requests` library to simulate client interactions with the server. 
- First, preload the tasks.
- Then, retrieve and print all tasks before updating.
- Send a **PUT** request to update the task by ID.
- Finally, retrieve and print all tasks after the update.

```python
import requests
URL = 'http://127.0.0.1:3000'

# Preload todos
response = requests.get(URL + '/preload/')

# Tasks before update
response = requests.get(URL + '/todos/')
print('Tasks before update:', response.json())

# Update the first task
response = requests.put(URL + '/todos/1/', json={'task': 'Buy almond milk'})
print(response.json())

# Tasks after update
response = requests.get(URL + '/todos/')
print('Tasks after update:', response.json())
```

### 5. **Running the Code**
Here’s how you would typically run the application:
1. **Django server**: Ensure your Django server is running using the command:
   ```bash
   python manage.py runserver
   ```

2. **Run the script**: Execute the script that sends requests to the Django server. The script will:
   - Preload initial data into the database.
   - Fetch and display the tasks before the update.
   - Send a PUT request to update the task description.
   - Fetch and display the updated tasks.

### Expected Output:
1. **Tasks before the update**:
   ```json
   {
       "todos": [
           {"id": 1, "task": "Buy milk", "completed": false},
           {"id": 2, "task": "Buy eggs", "completed": false},
           {"id": 3, "task": "Buy bread", "completed": true}
       ]
   }
   ```

2. **After the PUT request**:
   ```json
   {"message": "Todo updated"}
   ```

3. **Tasks after the update**:
   ```json
   {
       "todos": [
           {"id": 1, "task": "Buy almond milk", "completed": false},
           {"id": 2, "task": "Buy eggs", "completed": false},
           {"id": 3, "task": "Buy bread", "completed": true}
       ]
   }
   ```

This demonstrates how to effectively update records by ID using Django ORM and handle the entire workflow, from model definition to view logic and finally making API requests.

Ah, you're right! 😅 The `update_todo` function should indeed check for the presence of the `'task'` key before proceeding with the update. If the `'task'` key is missing from the request body, it should return an appropriate error response.

Here's an updated version of the function with that check included:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.views.decorators.http import require_http_methods
from django.shortcuts import get_object_or_404
from .models import Todo
import json

@csrf_exempt
@require_http_methods(['PUT'])
def update_todo(request, id):
    try:
        # Get the Todo item by ID
        todo = get_object_or_404(Todo, pk=id)

        # Parse the request body as JSON
        data = json.loads(request.body)

        # Check if 'task' is present in the request body
        if 'task' not in data:
            return JsonResponse({'error': "'task' field is required"}, status=400)

        # Update the task field and save the object
        todo.task = data['task']
        todo.save()

        # Return a success response
        return JsonResponse({'message': 'Todo updated'})
    
    except Exception as e:
        # Return an error response in case of any exception
        return JsonResponse({'error': str(e)}, status=400)
```

### Key Changes:
1. **Check for `'task'` key**: Before updating the `task` field, we check if the `'task'` key is present in the incoming request data. If it's missing, we return an error response.
   
   ```python
   if 'task' not in data:
       return JsonResponse({'error': "'task' field is required"}, status=400)
   ```

2. **Error Handling**: The function still returns a detailed error message in case of any exception, but now also handles missing `'task'` keys properly.

### Sample Error Response:
If the `'task'` key is missing, the function will return a response like this:
```json
{
    "error": "'task' field is required"
}
```

Would this version of the function meet your expectations? 😊

## Updating Todo Completion Status

To modify your existing code so that it updates the `completed` field of a Todo item instead of the `task` field, follow these steps:

1. **Update `update_todo` View:**
   Modify the `update_todo` view function to update the `completed` field based on the JSON data received in the PUT request.

   ```python
   @csrf_exempt
   @require_http_methods(['PUT']) # Only accept PUT requests
   def update_todo(request, id):
       try:
           todo = get_object_or_404(Todo, pk=id)
           data = json.loads(request.body)
           # Update the 'completed' field instead of 'task'
           if 'completed' in data:
               todo.completed = data['completed']
           todo.save()
           return JsonResponse({'message': 'Todo updated'})
       except Exception as e:
           return JsonResponse({'error': str(e)}, status=400)
   ```

2. **Adjust the Client-Side Code:**
   Ensure that your client-side code sends the correct JSON format to update the `completed` field:

   ```python
   import requests
   URL = 'http://127.0.0.1:3000'

   # Preload todos
   response = requests.get(URL + '/preload/')

   # Before updates
   response = requests.get(URL + '/todos/')
   print('Todos before updates:', response.json())

   # Update todo with id=1 to mark as completed
   response = requests.put(URL + '/todos/1/', json={'completed': True})
   print(response.json())

   # After updates
   response = requests.get(URL + '/todos/')
   print('Todos after updates:', response.json())
   ```

3. **URL Routing (`urls.py`):**
   Ensure your `urls.py` correctly routes requests to the `update_todo` view with the `id` parameter:

   ```python
   from django.urls import path
   from myapp import views

   urlpatterns = [
       path('todos/<int:id>/', views.update_todo, name='update_todo'),
       path('preload/', views.preload, name='preload'),
       path('todos/', views.get_todos, name='get_todos'),
   ]
   ```

With these modifications:
- The `update_todo` view now correctly updates the `completed` field of the specified Todo item based on the JSON data received.
- The client-side code sends requests to mark todos as completed by sending a JSON object with `{"completed": true}` to the respective endpoint (`/todos/<id>/`).
- Ensure your Django server and client-side scripts are configured and running correctly to see the expected behavior when marking todos as completed.

## Update Movies with Django ORM

Here's how to complete the `update_movie` view function based on your instructions. We'll follow the TODOs in the code and ensure the necessary checks and updates are in place.

### Updated `update_movie` View:

```python
@csrf_exempt
@require_http_methods(['PUT'])  # Only accept PUT requests
def update_movie(request, id):
    try:
        # Get the movie object by its ID
        movie = get_object_or_404(Movie, pk=id)

        # Parse the request body as JSON
        data = json.loads(request.body)

        # Check if 'title' and 'release_year' fields are present in the request data
        if 'title' not in data or 'release_year' not in data:
            return JsonResponse({'error': 'Both title and release_year fields are required'}, status=400)

        # Update the movie's title field
        movie.title = data['title']

        # Update the movie's release_year field
        movie.release_year = data['release_year']

        # Save the changes to the movie object
        movie.save()

        return JsonResponse({'message': 'Movie updated'})
    except Exception as e:
        return JsonResponse({'error': str(e)}, status=400)
```

### Explanation of the Changes:
1. **Fetching the Movie Object:**  
   We use `get_object_or_404(Movie, pk=id)` to retrieve the movie by its ID, or return a 404 error if the movie is not found.

2. **Parsing the Request Body:**  
   We parse the JSON request body using `json.loads(request.body)` to access the data sent in the PUT request.

3. **Validating Request Data:**  
   We check if both `title` and `release_year` are present in the request data. If either is missing, we return a JSON error response with a 400 status.

4. **Updating Fields:**  
   Once validated, we update both the `title` and `release_year` fields with the new values provided in the request.

5. **Saving the Changes:**  
   Finally, we save the updated movie object to the database using `movie.save()`.

### Client-Side Example:
This is how the client-side code will send the PUT request to update the movie's title and release year:

```python
import requests
URL = 'http://127.0.0.1:3000'

# Preload movies
response = requests.get(URL + '/preload/')

# Before update
response = requests.get(URL + '/movies/')
print('Movies before update:', response.json())

# Update movie with id=2
response = requests.put(URL + '/movies/2/', json={'title': 'The Matrix Reloaded', 'release_year': 2003})
print(response.json())

# After update
response = requests.get(URL + '/movies/')
print('Movies after update:', response.json())
```

### URLs Configuration (`urls.py`):
Ensure your `urls.py` has the correct routing for the update function:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('movies/<int:id>/', views.update_movie, name='update_movie'),
    path('preload/', views.preload, name='preload'),
    path('movies/', views.get_movies, name='get_movies'),
]
```

With these changes, you can now update a movie's title and release year using the `PUT` request. The validation ensures that both fields are required for the update to succeed.

## Update Movie Availability by Title

To complete the functionality in the `update_movie_availability` view, you need to retrieve the movie by its title, update its availability to `True`, and return an appropriate success message. Below is the completed code with the necessary changes.

### Updated `update_movie_availability` View:

```python
@csrf_exempt
@require_http_methods(['PUT'])  # Only accept PUT requests
def update_movie_availability(request):
    try:
        # Parse the request body as JSON
        data = json.loads(request.body)

        # Retrieve the movie object by title using the get_object_or_404 function
        movie = get_object_or_404(Movie, title=data['title'])

        # Update the movie's availability to True and save the object
        movie.availability = True
        movie.save()

        # Return a success message
        return JsonResponse({'message': 'Movie availability updated'})
    except Exception as e:
        return JsonResponse({'error': str(e)}, status=400)
```

### Explanation of the Changes:
1. **Retrieve Movie by Title:**  
   We use `get_object_or_404(Movie, title=data['title'])` to find the movie by its title. If the movie is not found, it returns a 404 error.

2. **Update Movie Availability:**  
   After retrieving the movie, we set `movie.availability = True` and then save the changes with `movie.save()`.

3. **Success Message:**  
   The response returns a success message `{'message': 'Movie availability updated'}` if the update is successful.

### Client-Side Code:
You can now test the endpoint by sending a `PUT` request to update a movie's availability by its title.

```python
import requests
URL = 'http://127.0.0.1:3000'

# Preload movies
response = requests.get(URL + '/preload/')

# Before update
response = requests.get(URL + '/movies/')
print('Before update:', response.json())

# Update movie availability (set availability to True for 'The Matrix')
response = requests.put(URL + '/update_movie_availability/', json={'title': 'The Matrix'})
print(response.json())

# After update
response = requests.get(URL + '/movies/')
print('After update:', response.json())
```

### URLs Configuration (`urls.py`):
Ensure the correct routing for the `update_movie_availability` function:

```python
from django.urls import path
from myapp import views

urlpatterns = [
    path('preload/', views.preload, name='preload'),
    path('update_movie_availability/', views.update_movie_availability, name='update_movie_availability'),
    path('movies/', views.get_movies, name='get_movies'),
]
```

### Model:
Your model already has the `availability` field, so no changes are needed here. The `Movie` model looks like this:

```python
from django.db import models

class Movie(models.Model):
    title = models.CharField(max_length=200)
    availability = models.BooleanField(default=False)

    def __str__(self):
        return self.title
```

With these changes, you now have an endpoint that updates the availability of a movie by its title. You can send a `PUT` request to `/update_movie_availability/` with the movie title to update its availability to `True`.